In [1]:
import h5py
import zarr
import sys
import napari
from abbott.h5_files import *
from abbott.conversions import *
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()

In [8]:
# convert zarr timelapse to h5 max projection 

fld = Path(r"Q:\carspi\Test")

out_fld = Path(r"Q:\carspi\Test") # change path to your share


def convert_zarr_to_h5(fld, out_fld, level):
    """
    Generate maximum projection h5 files from a folder containing timelapse zarr files. 
    
    parameters
    ----------
    fld: str or pathlib.Path
        folder containing your input data.
    out_fld: str or pathlib.Path
        folder where processed data will be saved 
    level: int
        downsampling level that should be used.
    """
    
    for fn in list(fld.glob('*.zarr')):
        print("processing {}...".format(fn.name))
        write_h5_file(fn, out_fld, level)

def write_h5_file(fn, out_fld, level):
    output_fn = out_fld / (fn.stem + '.h5') 
    output_fn.parent.mkdir(exist_ok=True, parents=True)
    f = zarr.open(fn, mode='r')
    fn_in= f[level][...] # we make max intensity projections on same level as model was trained on

    dsets = {}
    for idx, f in enumerate(np.rollaxis(fn_in, 1)):
        dset = []
        for tp in f:
            dset.append(np.max(tp, axis=0)) # convert stack to max projection
        dsets[f"ch_0{idx}"] = dset

    with h5py.File(output_fn, 'a') as f_out: # write to new h5 file
        for k,v in dsets.items():
            f_out.create_dataset(k, data=v)
            dset = f_out[k]
            dset.attrs['element_size_um'] = [3, 2.6, 2.6] # pixel size of z, y, x





convert_zarr_to_h5(fld, out_fld, 2)

processing ETiX_exp1_GT_IF1_s1.zarr...
processing ETiX_exp1_GT_IF1_s10.zarr...
processing ETiX_exp1_GT_IF1_s11.zarr...
processing ETiX_exp1_GT_IF1_s12.zarr...
processing ETiX_exp1_GT_IF1_s2.zarr...
